In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

project_dir = os.path.dirname(os.getcwd())
vehicles_folder_path = os.path.join(project_dir, 'vehicles')
nonvehicle_folder_path = os.path.join(project_dir, 'non-vehicles')

cars = []
notcars = []

def read_images(collection, parent_folder, folder_name):
    path = os.path.join(parent_folder, folder_name, '*.png')
    images = glob.glob(path)
    for image in images:
        collection.append(image)

def read_vehicle(folder_name):
    read_images(cars, vehicles_folder_path, folder_name)
    
def read_nonvehicle(folder_name):
    read_images(notcars, nonvehicle_folder_path, folder_name)

# Read cars images
read_vehicle('GTI_Far')
read_vehicle('GTI_Left')
read_vehicle('GTI_MiddleClose')
read_vehicle('GTI_Right')
read_vehicle('KITTI_extracted')
    
#read non-car images
read_nonvehicle('Extras')
read_nonvehicle('GTI')

In [2]:
import sys
sys.path.append('../')
from src.FeatureExtractor import FeatureExtractor

color_space = 'YCrCb'
hist_range = (0, 256)
spatial = 16
histbin = 32

t=time.time()
car_features = FeatureExtractor.extract_features_from_images(cars, cspace=color_space, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=hist_range)
notcar_features = FeatureExtractor.extract_features_from_images(notcars, cspace=color_space, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=hist_range)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract Color and Space features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using spatial binning of:',spatial,
    'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

11.06 Seconds to extract Color and Space features...
Using spatial binning of: 16 and 32 histogram bins
Feature vector length: 864
4.54 Seconds to train SVC...
Test Accuracy of SVC =  0.9561
My SVC predicts:  [ 0.  1.  1.  1.  0.  1.  0.  0.  0.  1.]
For these 10 labels:  [ 0.  1.  1.  1.  0.  1.  0.  0.  0.  1.]
0.001 Seconds to predict 10 labels with SVC


In [3]:
import pickle

to_save = {
    'svc': svc,
    'X_scaler': X_scaler,
    'features': (1, 1, 0), # color, space, hog
    'color_space': color_space,
    'spatial': spatial,
    'histbin': histbin,
    'hist_range': hist_range
}

# save transformer
with open(os.path.join(project_dir, 'sparse_color.pkl'), 'wb') as fid:
    pickle.dump(to_save, fid) 
print("pickle is saved")

pickle is saved


In [ ]:
import sys
sys.path.append('../')
from src.FeatureExtractor import FeatureExtractor

### TODO: Tweak these parameters and see how the results change.
colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 7
pix_per_cell = 4
cell_per_block = 4
hog_channel = 1 # Can be 0, 1, 2, or "ALL"

t=time.time()
car_features = FeatureExtractor.extract_hog_features(cars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = FeatureExtractor.extract_hog_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

In [ ]:
import pickle

to_save = {
    'svc': svc,
    'X_scaler': X_scaler,
    'features': (0, 0, 1), # color, space, hog
    'color_space': color_space,
    'orient': orient,
    'pix_per_cell': pix_per_cell,
    'cell_per_block': cell_per_block,
    'hog_channel': hog_channel
}

# save the classifier
with open(os.path.join(project_dir, 'hog.pkl'), 'wb') as fid:
    pickle.dump(to_save, fid) 